In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ---- Paths ----
ACTUAL_PATH = "/content/drive/MyDrive/Biology/data/actual.csv"
INDEP_PATH  = "/content/drive/MyDrive/Biology/data/data_set_ALL_AML_independent.csv"
TRAIN_PATH  = "/content/drive/MyDrive/Biology/data/data_set_ALL_AML_train.csv"

In [ ]:
# Load again (for clarity)
import pandas as pd

actual_df = pd.read_csv(ACTUAL_PATH)
train_df  = pd.read_csv(TRAIN_PATH)
indep_df  = pd.read_csv(INDEP_PATH)

print("Labels (actual.csv):")
print(actual_df.head(), "\n")

print("Training data:")
print(train_df.iloc[:5, :5], "\n")   # show first 5 rows × 5 cols

print("Independent data:")
print(indep_df.iloc[:5, :5])


In [ ]:
actual_df.head()

In [ ]:
train_df.head()

The training dataset represents microarray gene expression profiles for leukemia patients. Each row corresponds to one gene (or probe), identified by a Gene Description and a unique Gene Accession Number. The numerical columns (1, 2, 3, ...) represent individual patients, where each value indicates the measured expression level of that gene in that patient’s sample. For example, a value like -214 means that the probe detected lower-than-average expression for that gene in the corresponding patient, whereas a higher positive value indicates stronger expression.

In addition to numerical values, the dataset contains “call” columns (e.g., call, call.1, call.2). These are microarray quality-control flags that classify whether the gene is considered Present (P), Absent (A), or Marginal (M) in a particular patient. They help biologists check data reliability, but for computational modeling we usually drop these columns and focus only on the numeric expression levels.

# Data Cleaning & Reshaping

In [ ]:
# Drop any "call" columns (they are quality control flags, not expression values)
train_df_clean = train_df.drop(columns=[col for col in train_df.columns if "call" in col])
indep_df_clean = indep_df.drop(columns=[col for col in indep_df.columns if "call" in col])

print("Shape before cleaning:", train_df.shape)
print("Shape after cleaning:", train_df_clean.shape)

# Use Gene Accession Number as unique index

In [ ]:
# Set gene accession as index (gene IDs)
train_df_clean = train_df_clean.set_index("Gene Accession Number")
indep_df_clean = indep_df_clean.set_index("Gene Accession Number")

# Drop Gene Description (not numeric)
train_df_clean = train_df_clean.drop(columns=["Gene Description"])
indep_df_clean = indep_df_clean.drop(columns=["Gene Description"])

train_df_clean.head()

# Transpose so patients = rows, genes = columns

In [ ]:
# Transpose so that rows = patients, columns = genes
X_train = train_df_clean.T
X_indep = indep_df_clean.T

print("Train shape after transpose:", X_train.shape)
print("Independent shape after transpose:", X_indep.shape)

# Now each row = one patient, each column = one gene feature
X_train.head()

# Merge with Labels

In [ ]:
# Load labels (ALL vs AML)
labels = actual_df["cancer"].values

# Create dataframe with labels for train + independent sets
y = pd.Series(labels, name="Cancer_Type")

# Match shape
print("Number of patients:", len(y))
print("X_train shape + X_indep shape:", len(X_train) + len(X_indep))

Note: The labels file (actual.csv) has both train + independent patients. That means we will later merge them carefully so patient IDs align.

# Exploratory Data Analysis (EDA)

In [ ]:
# Class Distribution

# Count ALL vs AML patients
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="cancer", data=actual_df, palette="viridis")
plt.title("Distribution of Patients (ALL vs AML)", fontsize=14)
plt.show()

The class distribution plot shows that the dataset contains more patients with Acute Lymphoblastic Leukemia (ALL) compared to Acute Myeloid Leukemia (AML). This indicates a moderate imbalance in the data, which is important to note since it may influence downstream analyses and model performance. In practice, such an imbalance reflects biological reality, as ALL cases are generally more frequent than AML, especially in pediatric populations.

In [ ]:
# 2. Expression Value Ranges

# Visualize distribution of gene expression across all patients
X_all = pd.concat([X_train, X_indep], axis=0)

plt.figure(figsize=(8,5))
sns.histplot(X_all.values.flatten(), bins=100, kde=True, color="purple")
plt.title("Distribution of Gene Expression Values", fontsize=14)
plt.xlabel("Expression Level")
plt.ylabel("Frequency")
plt.show()

This plot shows the overall distribution of gene expression values across all patients. The majority of expression levels are concentrated very close to zero, forming a sharp peak, while a small number of genes show extremely high or low values, creating long tails on both sides. This indicates that most genes have relatively stable expression across samples, while a subset of genes exhibits strong variation, which may be biologically significant. Such heavy-tailed distributions are typical in microarray data and highlight the need for normalization and feature selection before applying machine learning.

In [ ]:
# 3. Variance Across Genes

# Find top 20 most variable genes
gene_variances = X_all.var().sort_values(ascending=False)[:20]

plt.figure(figsize=(12,5))
gene_variances.plot(kind="bar", color="teal")
plt.title("Top 20 Most Variable Genes", fontsize=14)
plt.ylabel("Variance in Expression")
plt.show()

This bar chart highlights the top genes with the highest variance in expression across patients. Genes with greater variance are more informative because they show stronger differences between individuals and may serve as potential biomarkers distinguishing ALL from AML. For example, probes such as M25079_s_at and HG1428:HT1428_s_at display the largest variability, suggesting that their activity levels differ widely between patients. In contrast, genes with low variance are relatively stable across all samples and contribute less to classification. Identifying these high-variance genes is a critical first step before performing feature selection or biological interpretation.

In [ ]:
# 4. Heatmap of Top Genes

# Select top 50 variable genes
top_genes = X_all[gene_variances.index[:50]]

plt.figure(figsize=(14,8))
sns.heatmap(top_genes.T, cmap="RdBu_r", center=0)
plt.title("Heatmap of Top 50 Variable Genes", fontsize=16)
plt.xlabel("Patients")
plt.ylabel("Genes")
plt.show()


This heatmap visualizes the expression patterns of the top variable genes across all patients. Each row corresponds to a gene, and each column represents a patient, with colors indicating expression intensity: red shades for higher expression and blue shades for lower expression. The alternating color patterns suggest that certain genes are consistently upregulated or downregulated in subsets of patients, which may correspond to the two leukemia subtypes (ALL vs AML). Such clustering is a strong indication that gene expression carries biologically meaningful signals that can differentiate between cancer types, making these genes potential biomarkers for classification.

In [ ]:
# 5. PCA Visualization

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=actual_df['cancer'], palette="Set2")
plt.title("PCA of Gene Expression (2 Components)", fontsize=14)
plt.xlabel("PC1 (%.2f%% variance)" % (pca.explained_variance_ratio_[0]*100))
plt.ylabel("PC2 (%.2f%% variance)" % (pca.explained_variance_ratio_[1]*100))
plt.show()


This scatter plot shows the PCA (Principal Component Analysis) projection of patient gene expression profiles into two dimensions. Each point represents a patient, with colors indicating their leukemia subtype (ALL vs AML). The first two principal components capture the largest sources of variation in the dataset, and we can observe that while there is some overlap, patients begin to form distinct clusters based on their cancer type. This separation suggests that gene expression patterns carry meaningful signals that differentiate ALL from AML, making PCA a powerful tool for visualizing the hidden structure of high-dimensional biological data.

In [ ]:
# 6. Volcano Plot (Gene-Level Differential Expression)

from scipy.stats import ttest_ind

# Separate ALL vs AML patients
all_patients = actual_df[actual_df['cancer']=="ALL"].index.values
aml_patients = actual_df[actual_df['cancer']=="AML"].index.values

X_all_aligned = X_all.reset_index(drop=True)
all_expr = X_all_aligned.iloc[all_patients]
aml_expr = X_all_aligned.iloc[aml_patients]

# Compute t-test for each gene
pvals = []
fold_changes = []

for gene in X_all.columns:
    t, p = ttest_ind(all_expr[gene], aml_expr[gene], equal_var=False)
    pvals.append(p)
    fold_changes.append(all_expr[gene].mean() - aml_expr[gene].mean())

# Volcano plot
plt.figure(figsize=(8,6))
sns.scatterplot(x=fold_changes, y=-np.log10(pvals), alpha=0.7)
plt.axhline(-np.log10(0.05), color="red", linestyle="--")
plt.title("Volcano Plot: Differential Gene Expression", fontsize=14)
plt.xlabel("Fold Change (ALL - AML)")
plt.ylabel("-log10(p-value)")
plt.show()


This plot combines statistical significance with expression differences to highlight genes that are most different between ALL and AML. The x-axis shows the fold change (difference in average expression between ALL and AML), while the y-axis shows the –log10(p-value) from a statistical test, meaning points higher up are more significant. Genes near the center have little difference between subtypes, whereas genes far to the left or right with high significance are strong candidate biomarkers. The red dashed line marks the threshold for statistical significance (p = 0.05). Overall, this plot reveals which genes are both highly variable and statistically reliable in distinguishing ALL from AML, making them prime targets for biological interpretation and downstream modeling.

In [ ]:
# We are going to extract the significant genes. We’ll filter by: p < 0.05 (significant) |fold change| > threshold (biologically relevant, e.g., >500)

import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

# Align ALL vs AML samples
all_patients = actual_df[actual_df['cancer']=="ALL"].index.values
aml_patients = actual_df[actual_df['cancer']=="AML"].index.values

X_all_aligned = X_all.reset_index(drop=True)
all_expr = X_all_aligned.iloc[all_patients]
aml_expr = X_all_aligned.iloc[aml_patients]

# Compute differential expression
pvals = []
fold_changes = []
genes = X_all.columns

for gene in genes:
    t, p = ttest_ind(all_expr[gene], aml_expr[gene], equal_var=False)
    pvals.append(p)
    fold_changes.append(all_expr[gene].mean() - aml_expr[gene].mean())

results_df = pd.DataFrame({
    "Gene": genes,
    "FoldChange": fold_changes,
    "pval": pvals
})
results_df["-log10p"] = -np.log10(results_df["pval"])

# Filter significant genes
sig_genes = results_df[(results_df["pval"] < 0.05) & (abs(results_df["FoldChange"]) > 500)]
sig_genes = sig_genes.sort_values("pval").head(10)  # top 10
sig_genes


In [ ]:
# 7. Clustering Patients

from sklearn.cluster import KMeans

# Try unsupervised clustering into 2 groups
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters, palette="coolwarm")
plt.title("Unsupervised Clustering of Patients", fontsize=14)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


Unlike the earlier PCA plot, this one uses class labels for coloring: one color for ALL and another for AML. We can see that patients naturally separate into two clusters, with AML cases (blue) grouping more distinctly away from ALL cases (orange). This clear separation indicates that the top principal components effectively capture biologically meaningful variation in gene expression, confirming that the dataset contains strong discriminative signals between the two leukemia subtypes.

# LLM Integration

In [ ]:
!pip install transformers accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
pip install sacremoses

# Load BioGPT

In [ ]:
model_name = "microsoft/biogpt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Query BioGPT Automatically

In [ ]:
def query_biogpt(gene_name, context="acute leukemia"):
    # Use a structured prompt so it doesn't echo
    prompt = f"Gene {gene_name} is associated with {context}. Its biological role is that"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=True,
        top_k=40,
        top_p=0.9,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Loop through top significant genes
for gene in sig_genes["Gene"].values:
    print(f"🔬 Gene: {gene}")
    print(query_biogpt(gene, context="acute leukemia"))
    print("-"*80)

# Run on Significant Genes

In [ ]:
# Run BioGPT on top genes from volcano plot
explanations = []

for gene in sig_genes["Gene"].values:
    response = query_biogpt(gene, context="acute leukemia")
    explanations.append({"Gene": gene, "Explanation": response})

bio_report = pd.DataFrame(explanations)
bio_report


In [ ]:
# Save outputs for later use (CSV or Markdown)
bio_report.to_csv("LLM_gene_explanations.csv", index=False)

with open("LLM_gene_explanations.md", "w") as f:
    for row in explanations:
        f.write(f"### 🔬 {row['Gene']}\n{row['Explanation']}\n\n")

# Pathway summarization

In [ ]:
query_biogpt("MYC, TP53, KRAS", context="cancer signaling pathways")


# Drug Target Prediction

In [ ]:
def drug_target_prediction(gene_list, context="acute leukemia"):
    genes = ", ".join(gene_list)

    # Force it into explanation mode with a continuation-style prompt
    prompt = f"""
Genes {genes} have been studied in {context}.
Potential drug targets associated with these genes include
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        top_k=40,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.2  # discourage repeating the prompt
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Example usage
drug_target_prediction(["M25079", "D63874"], context="acute leukemia")
